# Example

In [5]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

#load
df = pd.read_excel(r"C:\Users\Hannes\Desktop\wide.xlsx")
df[7:13]

,Year,US1724062096,US25383B1098,US2056842022,US46122E1055
7,1997,NaN,NaN,NaN,NaN
8,1998,NaN,NaN,NaN,NaN
9,1999,NaN,NaN,NaN,1.385043
10,2000,NaN,NaN,NaN,1.562616
11,2001,NaN,NaN,NaN,0.969316
12,2002,NaN,NaN,NaN,0.658200


In [11]:
df1 = pd.melt(df, id_vars=["Year"], var_name="FirmID", value_name="tobin")
df1.to_excel(r"C:\Users\Hannes\Desktop\long.xlsx", index=False)
df1

,Year,FirmID,tobin
0,1990,US1724062096,NaN
1,1991,US1724062096,NaN
2,1992,US1724062096,NaN
3,1993,US1724062096,NaN
4,1994,US1724062096,NaN
...,...,...,...
119,2016,US46122E1055,NaN
120,2017,US46122E1055,NaN
121,2018,US46122E1055,NaN
122,2019,US46122E1055,NaN


### tobinsq

#### wide to long

In [55]:
df = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Vars\TobinsQ_wide_panel.xlsx")
df.dropna(how="all", axis=1, inplace=True) #drop all empty columns

df_tobin = pd.melt(df, id_vars=["Year"], var_name="ISIN", value_name="FIRM_tobinsQ")
# df_tobin.to_excel(r"C:\Users\Hannes\Desktop\long.xlsx", index=False)

df_tobin.describe()

,Year,FIRM_tobinsQ
count,242044.000000,1.504040e+05
mean,2009.500000,3.133742e+01
std,6.344302,1.061629e+04
min,1999.000000,-9.298249e+03
25%,2004.000000,1.040980e+00
50%,2009.500000,1.381482e+00
75%,2015.000000,2.209847e+00
max,2020.000000,4.116593e+06


#### winsorize

In [56]:
df_tobin_winsorized = df_tobin.copy()

#set winsorize percentage (0.01, 0.025, 0.05 are common)
winsorize_percentile = 0.01
#create winsorized column
df_tobin_winsorized[f"FIRM_tobinsQ_w{str(winsorize_percentile)[-1:]}"] = df_tobin["FIRM_tobinsQ"].clip(lower=df_tobin["FIRM_tobinsQ"].quantile(winsorize_percentile), upper=df_tobin["FIRM_tobinsQ"].quantile(1-winsorize_percentile))

df_tobin_winsorized.describe()

,Year,FIRM_tobinsQ,FIRM_tobinsQ_w1
count,242044.000000,1.504040e+05,150404.000000
mean,2009.500000,3.133742e+01,2.119409
std,6.344302,1.061629e+04,2.218978
min,1999.000000,-9.298249e+03,0.466528
25%,2004.000000,1.040980e+00,1.040980
50%,2009.500000,1.381482e+00,1.381482
75%,2015.000000,2.209847e+00,2.209847
max,2020.000000,4.116593e+06,15.667197


In [73]:
df_tobin_winsorized

,Year,ISIN,FIRM_tobinsQ,FIRM_tobinsQ_w1
0,1999,US1724062096,NaN,NaN
1,2000,US1724062096,NaN,NaN
2,2001,US1724062096,NaN,NaN
3,2002,US1724062096,NaN,NaN
4,2003,US1724062096,9.568189,9.568189
...,...,...,...,...
242039,2016,IL0010943905,0.762619,0.762619
242040,2017,IL0010943905,0.890382,0.890382
242041,2018,IL0010943905,1.069413,1.069413
242042,2019,IL0010943905,1.160268,1.160268


### merge with Director-Firm-Years file

In [78]:
df_900 = pd.read_stata(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Stata\Director-Firm-Years\0118_Director-Firm-Years_CAAA.dta")

df_900["ISIN"].replace({"":np.nan}, inplace=True) #for some reason there are empty ISIN values
df_900.dropna(how="all", axis=0, inplace=True) #drop all empty columns
df_900.drop(columns=["tenure", "tenure_log"], inplace=True)
df_900

,DirectorID,FirmID,ISIN,Year,DIR_active,active,prior_vs_loss_dmy_all,prior_vs_loss_dmy_out,prior_gain_vs_event_dmy,years_before_gain,years_after_loss,DIR_directorships,DIR_directorships_change,DIR_max_gains,DIR_gainer,DIR_loser,DIR_changer,DIR_replace_happened,DIR_gain_happened,DIR_loss_happened,DIR_Age,DIR_Age_ln,DIR_DirectorNetworkSize,DIR_DirectorNetworkSize_ln,DIR_EPS_avg,DIR_ESG_Score_count,DIR_ESG_Score_sum,DIR_ESG_avg,DIR_ESG_avg_withoutEvent,DIR_GD_overall_avg,DIR_NED_board_size_avg,DIR_PhD,DIR_ROA_avg,DIR_accruals_change_avg,DIR_amort_intang_avg,DIR_attendance_level,DIR_awards,DIR_beta_avg,DIR_beta_avg_withoutEvent,DIR_beta_count,DIR_beta_sum,DIR_board_comp_avg_withoutEvent,DIR_board_comp_count,DIR_board_comp_sum,DIR_board_size_avg,DIR_ivy,DIR_management_departure_dmy,DIR_mcap_avg,DIR_mcap_total,DIR_mcap_total_withoutEvent,DIR_meetings_avg,DIR_meetings_avg_withoutEvent,DIR_meetings_count,DIR_meetings_sum,DIR_research_exp_avg,DIR_tobinsQ_avg,DIR_total_assets_avg,ESG_avg_delta,ESG_avg_delta_dmy,FIRM_EPS,FIRM_EPS_lagged,FIRM_ESG_Score,FIRM_ESG_controversies_score,FIRM_ROA,FIRM_ROA_w1,FIRM_ROA_w1_lagged,FIRM_SIC,FIRM_SIC1,FIRM_SIC2,FIRM_beta,FIRM_board_attendance,FIRM_board_attendance_lagged,FIRM_board_comp,FIRM_board_independence,FIRM_board_size,FIRM_cash,FIRM_closely_held_shares,FIRM_closely_held_shares_lagged,FIRM_debt_to_capital,FIRM_debt_to_capital_lagged,FIRM_growth_assets,FIRM_growth_assets_w1,FIRM_growth_assets_w1_lagged,FIRM_meetings,FIRM_total_assets,FIRM_vola,GD_overall_delta,InsideToOutsideRole,OutsideToInsideRole,beta_avg_delta,beta_avg_delta_dmy,beta_avg_delta_w1,beta_delta_rel_w1,board_comp_avg_delta,board_comp_avg_delta_dmy,board_comp_avg_delta_w1,board_comp_delta_rel_w1,committees_overlapping,mcap,mcap_w1,mcap_w1_ln,mcap_w1_lagged,mcap_w1_lagged_ln,mcap_delta,mcap_delta_dmy,mcap_delta_rel,mcap_delta_rel_w1,mcap_delta_w1,meetings_avg_delta,meetings_avg_delta_dmy,meetings_avg_delta_w1,meetings_delta_rel_w1,tookBreak,Committee_Audit,Committee_Chairman,Committee_Comp,Committee_Finance,Committee_Governance,Committee_Member,Committee_Nomination,Committee_OtherMember,Committee_Risk,female
0,2.169310e+05,2.897450e+08,US6369183024,1999.0,NaN,NaN,NaN,NaN,0.0,5.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,48.0,3.871201,4313.0,8.369389,0.205,0.0,NaN,NaN,NaN,NaN,0.0,0.0,8.68,10058.0,NaN,NaN,4.0,3.67,3.67,1.0,3.67,NaN,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,27313.0,1.808200,360158.0,NaN,NaN,NaN,NaN,NaN,NaN,6.38,6.38,NaN,6324.0,6.0,63.0,3.67,NaN,NaN,NaN,NaN,NaN,NaN,67.879997,NaN,0.010000,NaN,119.070000,119.070000,NaN,NaN,41763.0,0.4013,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2.169310e+05,2.897450e+08,US6369183024,2000.0,NaN,NaN,NaN,NaN,0.0,4.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,49.0,3.891820,4313.0,8.369389,0.252,0.0,NaN,NaN,NaN,NaN,0.0,0.0,10.59,1153.0,NaN,NaN,4.0,-1.48,-1.48,1.0,-1.48,NaN,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,28292.0,2.397679,335796.0,NaN,NaN,0.141,0.36,NaN,NaN,3.74,3.74,6.38,6324.0,6.0,63.0,-1.48,NaN,NaN,NaN,NaN,NaN,4543.0,69.470001,67.879997,14.490000,0.01,7.040000,7.040000,119.070000,NaN,44703.0,0.6181,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2.169310e+05,2.897450e+08,US6369183024,2001.0,NaN,NaN,NaN,NaN,0.0,3.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,50.0,3.912023,4313.0,8.369389,0.272,0.0,NaN,NaN,NaN,NaN,0.0,0.0,10.68,3484.0,5100.0,NaN,4.0,-0.94,-0.94,1.0,-0.94,NaN,0.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,28426.0,2.791824,368450.0,NaN,NaN,0.360,0.67,NaN,NaN,1.87,1.87,3.74,6324.0,6.0,63.0,-0.94,NaN,NaN,NaN,NaN,NaN,11259.0,66.500000,69.470001,11.550000,14.49,77.639999,77.639999,7.040000,NaN,79410.0,0.6721,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,2.169

#### tenure

In [79]:
df_firm_active = df_900[["DirectorID", "ISIN", "Year", "active"]]
df_active = df_firm_active[df_firm_active["active"].isna() == False].drop(columns=["active"]).reset_index(drop=True)

#count up duplicate year rows
# https://stackoverflow.com/questions/59334873/i-want-to-count-the-occurrence-of-duplicate-values-in-a-column-in-a-dataframe-an
df_active['tenure'] = df_active.groupby(['DirectorID','ISIN'], sort=False).cumcount()

df_active

,DirectorID,ISIN,Year,tenure
0,2.169310e+05,US6369183024,2004.0,0
1,2.169310e+05,US6369183024,2005.0,1
2,2.169310e+05,US6369183024,2006.0,2
3,2.169310e+05,US6369183024,2007.0,3
4,2.169310e+05,US46333X1081,2009.0,0
...,...,...,...,...
259686,2.065155e+11,VGG273581030,2018.0,1
259687,2.065155e+11,VGG273581030,2019.0,2
259688,2.068778e+11,KYG2345N1025,2017.0,0
259689,2.068778e+11,KYG2345N1025,2018.0,1


In [80]:
#tobinsq
df_900 = df_900.merge(df_tobin_winsorized, how="left", on=["ISIN", "Year"])

#generate lagged value by t-1
lagged_conditions = ( (df_900["FIRM_tobinsQ_w1"].shift(1).isna() == False) & 
    (df_900["DirectorID"] == df_900["DirectorID"].shift(1)) )
df_900["FIRM_tobinsQ_w1_lagged"] = np.where(lagged_conditions, df_900["FIRM_tobinsQ_w1"].shift(1), np.nan)

#move columns
for index, column in enumerate(["FIRM_tobinsQ", "FIRM_tobinsQ_w1", "FIRM_tobinsQ_w1_lagged"], start=1):
    #cut column
    col = df_900.pop(column)
    #insert column
    df_900.insert(df_900.columns.get_loc("FIRM_meetings") + index, col.name, col)


#tenure
df_900 = df_900.merge(df_active, how="left", on=["DirectorID", "ISIN", "Year"])
#move columns
col = df_900.pop("tenure")
#insert column
df_900.insert(df_900.columns.get_loc("active") + 1, col.name, col)


df_900[["DirectorID", "ISIN", "Year", "active", "tenure", "FIRM_tobinsQ", "FIRM_tobinsQ_w1", "FIRM_tobinsQ_w1_lagged"]]

,DirectorID,ISIN,Year,active,tenure,FIRM_tobinsQ,FIRM_tobinsQ_w1,FIRM_tobinsQ_w1_lagged
0,2.169310e+05,US6369183024,1999.0,NaN,NaN,NaN,NaN,NaN
1,2.169310e+05,US6369183024,2000.0,NaN,NaN,NaN,NaN,NaN
2,2.169310e+05,US6369183024,2001.0,NaN,NaN,NaN,NaN,NaN
3,2.169310e+05,US6369183024,2002.0,NaN,NaN,NaN,NaN,NaN
4,2.169310e+05,US6369183024,2003.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
916288,2.068778e+11,KYG2345N1025,2015.0,NaN,NaN,NaN,NaN,NaN
916289,2.068778e+11,KYG2345N1025,2016.0,NaN,NaN,NaN,NaN,NaN
916290,2.068778e+11,KYG2345N1025,2017.0,1.0,0.0,NaN,NaN,NaN
916291,2.068778e+11,KYG2345N1025,2018.0,1.0,1.0,NaN,NaN,NaN


In [84]:
df_900.to_stata(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Stata\Director-Firm-Years\0119_Director-Firm-Years_CAAA.dta", write_index=False)